In [148]:
from datetime import datetime, timedelta
from room_reservation.models import *

def check_room_is_booked(_room_id, _date):
    try:
        room = Room.objects.filter(pk=_room_id)
        if not len(room):
            return 'room id %s does not exist.' % (_room_id)

        bookings = Booking.objects.filter(rooms__pk=_room_id)
        if not len(bookings):
            return 'room no. %s is not booked at all.' % (room[0].room_no)

        check_date = datetime.strptime(_date, '%Y-%m-%d').date()

        for booking in bookings: 
            duration = booking.duration
            arrival_date = booking.arrival_date
            departure_date = arrival_date + timedelta(duration - 1)

            if arrival_date <= check_date and check_date <= departure_date:
                return 'room no.%s was reserved before in date %s.' % (room[0].room_no, check_date)

        return 'room no.%s in date %s is not booked.' % (room[0].room_no, check_date)
    except Exception as e:
        return str(e)
print(check_room_is_booked(77, '2023-03-09'))

room id 77 does not exist.


In [121]:
import pandas as pd
from django.db.models import Q, F
from datetime import datetime

def check_room_is_booked():
    try:
        _rooms = Room.objects.filter(Q(pk=3) | Q(pk=7))
        for room in _rooms:
            room_id = room.id
            print('room_id:', room_id)
            _bookings = Booking.objects.filter(rooms__pk=room_id).values('arrival_date', 'duration')
            if not len(_bookings):
                continue

            _duration = 2
            _date = '2023-03-09'

            start_date = datetime.strptime(_date, '%Y-%m-%d').date()
            end_date = start_date + timedelta(_duration - 1)
            date_range = pd.date_range(start_date, end_date)

            for booking in _bookings:
                arrival_date = booking['arrival_date']
                duration = booking['duration']
                departure_date = arrival_date + timedelta(duration - 1)
                print('arrival_date:', arrival_date)
                print('duration:', duration)
                print('departure_date:', departure_date)   

                for check_date in date_range:
                    check_date = check_date.date()
                    print('check_date:', check_date)

                    if arrival_date <= check_date and check_date <= departure_date:
                        print('room no. %s was reserved before in date "%s"' % (room.room_no, check_date))
                        return                

        print('-----ok') 
        return
    except Exception as e:
        print('error:', str(e))
        return
check_room_is_booked()

room_id: 3
room_id: 7
arrival_date: 2023-03-10
duration: 3
departure_date: 2023-03-12
check_date: 2023-03-09
check_date: 2023-03-10
room no. 7 was reserved before in date "2023-03-10"
